# Captchas Solver

### Toolset
- Python3
- OpenCV
- Keras
- Tensorflow

In [13]:
import os
import os.path
import cv2
import glob
import imutils

import pickle
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from imutils import paths

from keras.models import load_model

import matplotlib.pyplot as plt

### Building and Training the Neural Network

In [14]:
def resize_to_fit(image, width, height):
    """
    A helper function to resize an image to fit within a given size
    :param image: image to resize
    :param width: desired width in pixels
    :param height: desired height in pixels
    :return: the resized image
    """

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image

- get each image and tranform it to gray and to scale 20x20 pixels
- extend each image to have a 3rd dimension (dimension of colors that will not be used in this case)
- get the label of each image (saved on the path name)

- having data and labels:
    - split the data into training and test sets
    - convert lables to one-hot enconding to do multi-class classification
    - save model that encodes labels into one-hot encondings to later preprocess new observations

In [15]:
data = []
labels = []

# loop over the input images
for image_file in paths.list_images('extracted_letter_images'):
    # Load the image and convert it to grayscale
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the letter so it fits in a 20x20 pixel box
    image = resize_to_fit(image, 20, 20)

    # Add a third channel dimension to the image to make Keras work
    image = np.expand_dims(image, axis=2)

    # Grab the name of the letter based on the folder it was in
    label = image_file.split(os.path.sep)[-2]

    # Add the letter image and it's label to our training data
    data.append(image)
    labels.append(label)

# data[0].shape -> (20, 20, 1)

# Scale the raw pixel intensities to [0,1] -> improves training
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.25, random_state=0, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=0, stratify=y_train)

# Convert the labels (letters) into one-hot-encodings that Keras can work with
# Used to extend classifiers to multi-class classifiers. It will make the model consisting of "one model per class
lb = LabelBinarizer().fit(y_train)
y_train = lb.transform(y_train)
y_val = lb.transform(y_val)
y_test = lb.transform(y_test)

# Save mapping from labels to one-hot-encodings
with open("simple_labels.dat", 'wb') as f:
    pickle.dump(lb, f)


- Model with 2 convultional layers (20 neurons + 50 neurons) with relu activation functions

- Check what MaxPooling is

- Flatten the output of the second convultional layer to make it as input of a dense fully connected layer

- Ass 2 more dense layers, the latter as output layer with 32 neurons (same numnber as output characters possibilities) and softmax activation

- Compile model with cross-entropy loss and Adam optimizer

- Fit the model to the training data and use test data for validation

- Save the model for later use to classifying new data


In [16]:
model = Sequential()

# 1st convolutional layer with max pooling
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# 2nd convolutional layer with max pooling
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))

# Output layer with 32 nodes -> number of possible characters
model.add(Dense(32, activation = 'softmax'))

# Ask Keras to build the model with TensorFlow behind the scenes
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics =['accuracy'])

# # Train the neural network
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size = 32, epochs = 10, verbose = 1)

model.save('simple_captcha_model.hdf5')

Epoch 1/10
682/682 [==============================] - 56s 81ms/step - loss: 0.2482 - accuracy: 0.9396 - val_loss: 0.0275 - val_accuracy: 0.9935
Epoch 2/10
682/682 [==============================] - 52s 77ms/step - loss: 0.0146 - accuracy: 0.9964 - val_loss: 0.0119 - val_accuracy: 0.9977
Epoch 3/10
682/682 [==============================] - 49s 73ms/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.0116 - val_accuracy: 0.9964
Epoch 4/10
682/682 [==============================] - 52s 76ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 0.0097 - val_accuracy: 0.9982
Epoch 5/10
682/682 [==============================] - 49s 72ms/step - loss: 0.0022 - accuracy: 0.9992 - val_loss: 0.0065 - val_accuracy: 0.9985
Epoch 6/10
682/682 [==============================] - 53s 77ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.0114 - val_accuracy: 0.9982
Epoch 7/10
682/682 [==============================] - 50s 73ms/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 0.0067 - val_accuracy:

### Model evaluation

In [17]:
results = model.evaluate(X_test, y_test, batch_size=32)
results

303/303 [==============================] - 3s 9ms/step - loss: 0.0143 - accuracy: 0.9975


[0.014310354366898537, 0.9975221753120422]

### Seing the model in action

In [18]:
def simple_captcha_test_preprocessing(image_file):

    # Load image
    img = cv2.imread(image_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.copyMakeBorder(img, 20, 20, 20, 20, cv2.BORDER_REPLICATE)

    img_thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    img_contours = cv2.findContours(img_thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_contours = img_contours[1] if imutils.is_cv3() else img_contours[0]

    letter_image_regions = []

    for contour in img_contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        if w / h > 1.25:
            half_width = int(w/2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            letter_image_regions.append((x, y, w, h))

    if len(letter_image_regions) != 4:
        return None

    letter_image_regions = sorted(letter_image_regions, key= lambda x: x[0])

    return (img, letter_image_regions)

In [19]:
# load label enconder
with open('simple_labels.dat', 'rb') as f:
    lb = pickle.load(f)

# load CNN
model = load_model('simple_captcha_model.hdf5')

# grab images to test against
captcha_image_files = list(paths.list_images('generated_captcha_images'))
captcha_image_files = np.random.choice(captcha_image_files, size=(10,), replace=False)

In [20]:
for image_file in captcha_image_files:

    preprocess_return = simple_captcha_test_preprocessing(image_file)

    if preprocess_return == None:
        continue
    
    (image, letter_image_regions) = preprocess_return

    output = cv2.merge([image] * 3)
    predictions = []

    for letter_bounding_box in letter_image_regions:
        (x, y, w, h) = letter_bounding_box

        letter_image = image[y-2 : y + h + 2, x-2 : x+w+2]

        letter_image = resize_to_fit(letter_image, 20, 20)

        letter_image = np.expand_dims(letter_image, axis = 2)
        letter_image = np.expand_dims(letter_image, axis = 0)
        
        prediction = model.predict(letter_image)

        letter = lb.inverse_transform(prediction)[0]
        predictions.append(letter)

        cv2.rectangle(output, (x-2, y-2), (x+w+4, y+h+4), (0, 255, 0), 1)
        cv2.putText(output, letter, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)

    captcha_text = "".join(predictions)
    print("CAPTCHA text is {}".format(captcha_text))

    cv2.imshow("Output", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 


1/1 [==============================] - 0s 69ms/step
CAPTCHA text is 7CNG
1/1 [==============================] - 0s 28ms/step
CAPTCHA text is 2E92
1/1 [==============================] - 0s 31ms/step
CAPTCHA text is JJ24
1/1 [==============================] - 0s 21ms/step
CAPTCHA text is 4KMU
1/1 [==============================] - 0s 22ms/step
CAPTCHA text is FV8X
1/1 [==============================] - 0s 33ms/step
CAPTCHA text is 3XYE
1/1 [==============================] - 0s 29ms/step
CAPTCHA text is ZSHB
1/1 [==============================] - 0s 20ms/step
CAPTCHA text is BMRE
1/1 [==============================] - 0s 26ms/step
CAPTCHA text is VEPM
1/1 [==============================] - 0s 21ms/step
CAPTCHA text is 5LTB
